## Livrable pour le projet de SDP

In [3]:
import gurobipy as gp
from gurobipy import GRB

## Données du problème initial

In [20]:
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic Pathology", "Génétique"]
poids = [8, 7, 7, 6, 6, 5, 6]

# Notes des candidats 
notes_x = [85, 81, 71, 69, 75, 81, 88] # Xavier
notes_y = [81, 81, 75, 63, 67, 88, 95] # Yvonne
notes_z = [74, 89, 74, 81, 68, 84, 79]
notes_t = [74, 71, 84, 91, 77, 76, 73]
notes_w = [79, 69, 78, 76, 67, 84, 79]
notes_w_prime = [57, 76, 81, 76, 82, 86, 77]
notes_u = [72, 66, 75, 85, 88, 66, 93]
notes_v = [71, 73, 63, 92, 76, 79, 93]

# Autres candidats
notes_a1 = [89, 74, 81, 68, 84, 79, 77]
notes_a2 = [71, 84, 91, 79, 78, 73.5, 77] # 73.5 pour Forensic

## Question 1: Modèle 1-1

Calcul des contributions pondérées: La contribution est : poids * (note_x - note_y)

In [4]:
contributions = []
for i in range(len(matieres)):
    delta = notes_x[i] - notes_y[i]
    contrib = poids[i] * delta
    contributions.append(contrib)

print("Calcul des contributions par matière :")
for m, c in zip(matieres, contributions):
    print(f"{m}: {c:+}")

# Vérification avec le tableau de la page 2[cite: 30]:
# A (Anatomie): +32, C (Chirurgie): -28, etc.

Calcul des contributions par matière :
Anatomie: +32
Biologie: +0
Chirurgie: -28
Diagnostic: +36
Epidemiologie: +48
Forensic Pathology: -35
Génétique: -42


In [5]:
# 3. Identification des ensembles Pros (P) et Cons (C)
P_indices = [i for i, c in enumerate(contributions) if c > 0] # Indices des matières "Pour"
C_indices = [j for j, c in enumerate(contributions) if c < 0] # Indices des matières "Contre"

print(f"\nEnsemble Pros (Indices) : {P_indices} -> {[matieres[i] for i in P_indices]}")
print(f"Ensemble Cons (Indices) : {C_indices} -> {[matieres[j] for j in C_indices]}")

# Calcul de la matrice de validité des trade-offs (1-1)
# Un trade-off (i, j) est valide si contribution[i] + contribution[j] > 0 [cite: 31]
valid_pairs = []
for i in P_indices:
    for j in C_indices:
        if contributions[i] + contributions[j] > 0:
            valid_pairs.append((i, j))

print(f"\nNombre de paires (1-1) valides identifiées : {len(valid_pairs)}")


Ensemble Pros (Indices) : [0, 3, 4] -> ['Anatomie', 'Diagnostic', 'Epidemiologie']
Ensemble Cons (Indices) : [2, 5, 6] -> ['Chirurgie', 'Forensic Pathology', 'Génétique']

Nombre de paires (1-1) valides identifiées : 6


In [6]:
# 4. Création du modèle d'optimisation
m = gp.Model("Explication_X_sup_Y_Type_1_1")

# Variables de décision : x[i,j] = 1 si on utilise la matière i pour compenser la matière j
# On ne crée les variables QUE pour les paires valides pour réduire l'espace de recherche
x = {}
for i, j in valid_pairs:
    x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"pair_{matieres[i]}_{matieres[j]}")

# Objectif : Maximiser le nombre de critères "Contre" expliqués (couverts)
# Max sum(x_ij)
m.setObjective(gp.quicksum(x[i, j] for i, j in valid_pairs), GRB.MAXIMIZE)

# Contraintes

# C1: Chaque élément "Contre" (j) doit être couvert exactement une fois, pour former 
# l'ensemble cons(x,y) à la fin
# sum_{i in P} x_ij == 1  pour tout j in C
for j in C_indices:
    # On somme sur tous les i qui forment une paire valide avec ce j
    m.addConstr(gp.quicksum(x[i, j] for i in P_indices if (i, j) in valid_pairs) == 1, name=f"cover_con_{matieres[j]}")

# C2: Chaque élément "Pour" (i) ne peut être utilisé qu'une seule fois au maximum
# sum_{j in C} x_ij <= 1 pour tout i in P
for i in P_indices:
    # On somme sur tous les j qui forment une paire valide avec ce i
    m.addConstr(gp.quicksum(x[i, j] for j in C_indices if (i, j) in valid_pairs) <= 1, name=f"use_pro_{matieres[i]}")

# 5. Résolution
m.optimize()

Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 6 rows, 6 columns and 12 nonzeros (Max)
Model fingerprint: 0xa9562452
Model has 6 linear objective coefficients
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 18 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.0000000000

## Question 2: Modèle (1-m)

In [12]:
def solve_explanation_1_m(notes_winner, notes_loser, poids, matieres, epsilon=1e-4):
    """
    Résout le problème d'explication de type (1-m).
    Retourne le statut et l'explication si trouvée.
    """
    # 1. Calcul des contributions
    deltas = []
    for k in range(len(matieres)):
        val = poids[k] * (notes_winner[k] - notes_loser[k])
        deltas.append(val)

    # 2. Séparation Pros / Cons
    P = [i for i, d in enumerate(deltas) if d > 0]
    C = [j for j, d in enumerate(deltas) if d < 0]
    
    print(f"Pros (Indices): {P} -> {[matieres[i] for i in P]} (Val: {[deltas[i] for i in P]})")
    print(f"Cons (Indices): {C} -> {[matieres[j] for j in C]} (Val: {[deltas[j] for j in C]})")

    if not C:
        return "Trivial (Pas de Cons)", None
    
    # 3. Modèle Gurobi
    m = gp.Model("Explication_1_m")
    m.setParam('OutputFlag', 0) # Mode silencieux

    # Variables
    # x[i,j] = 1 si Pro i couvre Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"link_{i}_{j}")
    
    # y[i] = 1 si Pro i est utilisé
    y = {}
    for i in P:
        y[i] = m.addVar(vtype=GRB.BINARY, name=f"use_{i}")

    # Contrainte C1: Chaque Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x[i, j] for i in P) == 1, name=f"cover_{j}")

    # Contrainte C2: Lien x_ij <= y_i
    for i in P:
        for j in C:
            m.addConstr(x[i, j] <= y[i], name=f"link_logic_{i}_{j}")

    # Contrainte C3: Somme pondérée positive pour chaque groupe (1-m)
    # delta_i * y_i + sum(delta_j * x_ij) >= epsilon * y_i
    for i in P:
        sum_cons = gp.quicksum(deltas[j] * x[i, j] for j in C)
        m.addConstr(deltas[i] * y[i] + sum_cons >= epsilon * y[i], name=f"validity_{i}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        explanation = {}
        for i in P:
            if y[i].X > 0.5:
                covered_cons = [matieres[j] for j in C if x[i, j].X > 0.5]
                explanation[matieres[i]] = covered_cons
        return "OPTIMAL", explanation
    elif m.Status == GRB.INFEASIBLE:
        return "INFEASIBLE", None
    else:
        return f"Status {m.Status}", None

In [13]:
print("--- Test de l'explication (1-m) pour w > w' ---")
status, result = solve_explanation_1_m(notes_w, notes_w_prime, poids, matieres)

print("-" * 30)
if status == "OPTIMAL":
    print("Succès ! Explication trouvée :")
    for pro, cons in result.items():
        print(f" - {pro} compense {cons}")
elif status == "INFEASIBLE":
    print("Résultat : INFEASIBLE")
    print("Preuve : Il n'existe pas d'explication de type (1-m) pour w > w'.")



print("--- Test de l'explication (1-m) pour u > v ---")
status, result = solve_explanation_1_m(notes_u, notes_v, poids, matieres)

print("-" * 30)
if status == "OPTIMAL":
    print("Succès ! Explication trouvée :")
    for pro, cons in result.items():
        print(f" - {pro} compense {cons}")
elif status == "INFEASIBLE":
    print("Résultat : INFEASIBLE")
    print("Preuve : Il n'existe pas d'explication de type (1-m) pour u > v.")

--- Test de l'explication (1-m) pour w > w' ---
Pros (Indices): [0, 6] -> ['Anatomie', 'Génétique'] (Val: [176, 12])
Cons (Indices): [1, 2, 4, 5] -> ['Biologie', 'Chirurgie', 'Epidemiologie', 'Forensic Pathology'] (Val: [-49, -21, -90, -10])
------------------------------
Succès ! Explication trouvée :
 - Anatomie compense ['Biologie', 'Chirurgie', 'Epidemiologie', 'Forensic Pathology']
 - Génétique compense []
--- Test de l'explication (1-m) pour u > v ---
Pros (Indices): [0, 2, 4] -> ['Anatomie', 'Chirurgie', 'Epidemiologie'] (Val: [8, 84, 72])
Cons (Indices): [1, 3, 5] -> ['Biologie', 'Diagnostic', 'Forensic Pathology'] (Val: [-49, -42, -65])
------------------------------
Résultat : INFEASIBLE
Preuve : Il n'existe pas d'explication de type (1-m) pour u > v.


## Question 3: Modèle (m-1)

In [17]:
import gurobipy as gp
from gurobipy import GRB

def solve_explanation_m_1(candidate_sup, candidate_inf, notes_sup, notes_inf, poids, matieres):
    """
    Résout le problème d'explication de type (m-1).
    Plusieurs 'Pros' peuvent compenser un seul 'Con'.
    """
    # 1. Calcul des deltas
    deltas = []
    for k in range(len(matieres)):
        val = poids[k] * (notes_sup[k] - notes_inf[k])
        deltas.append(val)
        
    P = [i for i, d in enumerate(deltas) if d > 0]
    C = [j for j, d in enumerate(deltas) if d < 0]
    
    print(f"\nComparaison {candidate_sup} > {candidate_inf}")
    print(f"Pros: {[(matieres[i], deltas[i]) for i in P]}")
    print(f"Cons: {[(matieres[j], deltas[j]) for j in C]}")

    if not C:
        print("Aucun désavantage à expliquer.")
        return

    # 2. Modèle Gurobi
    m = gp.Model(f"Explication_m_1_{candidate_sup}_{candidate_inf}")
    m.setParam('OutputFlag', 0)

    # Variables x[i, j] : Pro i aide à couvrir Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{matieres[i]}_{matieres[j]}")

    # C1: Chaque Pro utilisé au max 1 fois
    for i in P:
        m.addConstr(gp.quicksum(x[i, j] for j in C) <= 1, name=f"disjoint_{matieres[i]}")

    # C2: Chaque Con doit être compensé (Somme Pros + Con >= 0)
    # On utilise une tolérance très faible ou nulle. Ici >= 0 accepte l'égalité parfaite.
    for j in C:
        m.addConstr(gp.quicksum(deltas[i] * x[i, j] for i in P) + deltas[j] >= 0, 
                    name=f"cover_{matieres[j]}")

    # 3. Résolution
    m.optimize()

    # 4. Analyse
    if m.Status == GRB.OPTIMAL:
        print(">>> SUCCÈS : Explication (m-1) trouvée.")
        for j in C:
            # Récupérer les Pros assignés à ce Con j
            assigned_pros = [matieres[i] for i in P if x[i, j].X > 0.5]
            val_pros = sum(deltas[i] for i in P if x[i, j].X > 0.5)
            print(f"  - Le désavantage '{matieres[j]}' ({deltas[j]}) est compensé par {assigned_pros} (Total: +{val_pros})")
    elif m.Status == GRB.INFEASIBLE:
        print(">>> ÉCHEC : Pas d'explication (m-1) possible.")
    else:
        print(f"Statut solveur : {m.Status}")

In [18]:
# Test 1 : y > z
solve_explanation_m_1("y", "z", notes_y, notes_z, poids, matieres)

# Test 2 : z > t
solve_explanation_m_1("z", "t", notes_z, notes_t, poids, matieres)


Comparaison y > z
Pros: [('Anatomie', 56), ('Chirurgie', 7), ('Forensic Pathology', 20), ('Génétique', 96)]
Cons: [('Biologie', -56), ('Diagnostic', -108), ('Epidemiologie', -6)]
>>> SUCCÈS : Explication (m-1) trouvée.
  - Le désavantage 'Biologie' (-56) est compensé par ['Anatomie'] (Total: +56)
  - Le désavantage 'Diagnostic' (-108) est compensé par ['Forensic Pathology', 'Génétique'] (Total: +116)
  - Le désavantage 'Epidemiologie' (-6) est compensé par ['Chirurgie'] (Total: +7)

Comparaison z > t
Pros: [('Biologie', 126), ('Forensic Pathology', 40), ('Génétique', 36)]
Cons: [('Chirurgie', -70), ('Diagnostic', -60), ('Epidemiologie', -54)]
>>> ÉCHEC : Pas d'explication (m-1) possible.


## Question 4: Modèle Hybride

In [ ]:
def solve_mixed_explanation(name_winner, name_loser, notes_winner, notes_loser, weights, subjects):
    """
    Résout le problème d'explication mixte : autorise les structures (1-m) OU (m-1).
    """
    # 1. Calcul des contributions
    deltas = []
    for k in range(len(subjects)):
        d = weights[k] * (notes_winner[k] - notes_loser[k])
        deltas.append(d)
        
    P_indices = [i for i, d in enumerate(deltas) if d > 0]
    C_indices = [j for j, d in enumerate(deltas) if d < 0]
    
    print(f"\n=== Comparaison {name_winner} > {name_loser} ===")
    print(f"Pros: {[(subjects[i], deltas[i]) for i in P_indices]}")
    print(f"Cons: {[(subjects[j], deltas[j]) for j in C_indices]}")
    
    if not C_indices:
        print("Aucun point négatif à expliquer.")
        return

    # 2. Modèle
    m = gp.Model("Mixed_Explanation")
    m.setParam('OutputFlag', 0)
    
    # --- Variables ---
    # Pour le type (1-m) : Pro i est le "Hub" qui couvre les Cons j (Leaves)
    # x_1m[i,j] = 1 si i couvre j dans un schéma (1-m)
    x_1m = {} 
    
    # Pour le type (m-1) : Con j est le "Hub" couvert par les Pros i (Leaves)
    # x_m1[i,j] = 1 si i aide à couvrir j dans un schéma (m-1)
    x_m1 = {}

    for i in P_indices:
        for j in C_indices:
            x_1m[i, j] = m.addVar(vtype=GRB.BINARY, name=f"1m_{subjects[i]}_{subjects[j]}")
            x_m1[i, j] = m.addVar(vtype=GRB.BINARY, name=f"m1_{subjects[i]}_{subjects[j]}")

    # Indicateurs de rôle "Hub"
    # h_P[i] = 1 si le Pro i est utilisé comme centre d'un (1-m)
    h_P = {i: m.addVar(vtype=GRB.BINARY) for i in P_indices}
    
    # h_C[j] = 1 si le Con j est utilisé comme centre d'un (m-1)
    h_C = {j: m.addVar(vtype=GRB.BINARY) for j in C_indices}

    # --- Contraintes ---

    # 1. Partition des Cons (Tout Con doit être couvert exactement une fois)
    # Un Con j est soit une "feuille" d'un (1-m), soit un "hub" d'un (m-1)
    for j in C_indices:
        # Somme des liens entrants (1-m) + Est-ce un hub (m-1) ?
        m.addConstr(gp.quicksum(x_1m[i, j] for i in P_indices) + h_C[j] == 1, name=f"Cover_{j}")

    # 2. Disjonction des Pros (Tout Pro utilisé au max une fois)
    # Un Pro i est soit un "hub" d'un (1-m), soit une "feuille" d'un (m-1), soit inutilisé
    for i in P_indices:
        # Est-ce un hub (1-m) + Somme des liens sortants (m-1)
        m.addConstr(h_P[i] + gp.quicksum(x_m1[i, j] for j in C_indices) <= 1, name=f"Use_{i}")

    # 3. Cohérence logique des liens
    # Si lien (1-m) i->j existe, alors i doit être déclaré Hub
    for i in P_indices:
        for j in C_indices:
            m.addConstr(x_1m[i, j] <= h_P[i])
            
    # Si lien (m-1) i->j existe, alors j doit être déclaré Hub
    for i in P_indices:
        for j in C_indices:
            m.addConstr(x_m1[i, j] <= h_C[j])

    # 4. Validité Financière (Somme pondérée > 0)
    
    # Pour chaque Pro Hub (Structure 1-m)
    # delta_i + sum(delta_j * x_1m_ij) >= 0
    for i in P_indices:
        contrib_cons = gp.quicksum(deltas[j] * x_1m[i, j] for j in C_indices)
        # On multiplie par h_P[i] pour désactiver la contrainte si pas hub (big-M ou juste logique 0>=0)
        # Ici : delta_i * h_P[i] + contrib_cons >= 0 (car x_ij est 0 si h_P est 0)
        m.addConstr(deltas[i] * h_P[i] + contrib_cons >= 0, name=f"Valid_1m_{i}")

    # Pour chaque Con Hub (Structure m-1)
    # sum(delta_i * x_m1_ij) + delta_j >= 0
    for j in C_indices:
        contrib_pros = gp.quicksum(deltas[i] * x_m1[i, j] for i in P_indices)
        m.addConstr(contrib_pros + deltas[j] * h_C[j] >= 0, name=f"Valid_m1_{j}")

    # Résolution
    m.optimize()

    # --- Affichage ---
    if m.Status == GRB.OPTIMAL:
        print(">>> SUCCÈS : Explication mixte trouvée !")
        print("Détails :")
        
        # Affichage des structures (1-m)
        for i in P_indices:
            if h_P[i].X > 0.5:
                cons_covered = [subjects[j] for j in C_indices if x_1m[i, j].X > 0.5]
                val_cons = sum(deltas[j] for j in C_indices if x_1m[i, j].X > 0.5)
                print(f"  [Type 1-m] L'avantage {subjects[i]} (+{deltas[i]}) compense {cons_covered} ({val_cons})")

        # Affichage des structures (m-1)
        for j in C_indices:
            if h_C[j].X > 0.5:
                pros_used = [subjects[i] for i in P_indices if x_m1[i, j].X > 0.5]
                val_pros = sum(deltas[i] for i in P_indices if x_m1[i, j].X > 0.5)
                print(f"  [Type m-1] Les avantages {pros_used} (+{val_pros}) compensent {subjects[j]} ({deltas[j]})")
                
    elif m.Status == GRB.INFEASIBLE:
        print(">>> ÉCHEC : Impossible de trouver une explication mixte valide.")
    else:
        print(f"Statut : {m.Status}")



In [ ]:
solve_mixed_explanation("z", "t", notes_z, notes_t, poids, matieres)

# Cas 2 : a1 > a2 (Nouveaux candidats)
solve_mixed_explanation("a1", "a2", notes_a1, notes_a2, poids, matieres)